In [ ]:
!pip install -U tqdm gdown

In [18]:
import os
import shutil

import gdown
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import ResNet50_Weights
from tqdm.auto import tqdm

print('Dependencies loaded.')

print("===================================================")
if(torch.cuda.is_available()):
    device = torch.device("cuda")
    print('Cuda available: {}'.format(torch.cuda.is_available()))
    print("GPU: " + torch.cuda.get_device_name(torch.cuda.current_device()))
    print("Total memory: {:.1f} GB".format((float(torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)))))
else:
    device = torch.device("cpu")
    print('Cuda not available, so using CPU. Please consider switching to a GPU runtime!')
print("===================================================")

Dependencies loaded

Found GPU: cuda


In [ ]:
# Dataset Download
ID = '1wYzBpucuU_RPMebUrWi2TH3Mfz-ykBGl'
output = 'Dataset.zip'

datasetDirectory = os.path.join("Dataset", "paperDataset")
if not (os.path.exists(datasetDirectory)):
  os.makedirs(datasetDirectory)

dataset_url = 'https://drive.google.com/uc?id=' + ID + '&export=download&confirm=t'
gdown.download(dataset_url, output, quiet=False)

print('Extracting the zipped dataset..')
shutil.unpack_archive("Dataset.zip", datasetDirectory)

print('Deleting the previous downloaded zip.')
os.remove('Dataset.zip')
print('Done.')

In [2]:
model = models.resnet50(weights=ResNet50_Weights.DEFAULT)

# Replace the last fully connected layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # 2 for binary classification

# Freeze pre-trained layers
for n, p in model.named_parameters():
    if(n.startswith("layer4") or n.startswith("fc")):
        p.requires_grad = True
    else:
        p.requires_grad = False
    #print(F"Parameter: {n} | Requires grad: {p.requires_grad}")

model = model.to(device)

c:\Users\gianc\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\gianc\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)  # Optimizer for training the last layer
epochs = 10

In [ ]:
# Define data transformations
transformTrain = transforms.Compose([
    transforms.RandomHorizontalFlip(0.5),
    transforms.CenterCrop(286),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transformsTest = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define paths for train_dir and test_dir according to split perfomed on OFs dataset
train_dir = os.path.join("Dataset", "paperDataset", "train")
test_dir = os.path.join("Dataset", "paperDataset", "test")

# Load the custom dataset
train_dataset = ImageFolder(root=train_dir, transform=transformTrain)
test_dataset = ImageFolder(root=test_dir, transform=transformsTest)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Dataloaders
train_features, train_labels = next(iter(train_loader))
print("===================================================")
print(f"Training feature shape : {train_features.size()}")
print(f"Training labels shape  : {train_labels.size()}")
print('----------------------------------------------')
test_features, test_labels = next(iter(test_loader))
print(f"Test feature shape     : {test_features.size()}")
print(f"Test labels shape      : {test_labels.size()}")
print("===================================================")

image = train_features[0]
plt.imshow(image.permute(1,2,0).clamp(0,1))
plt.title(F"Label: {train_labels[0]}")
plt.show()

In [ ]:
trainLosses = []
trainAccuracies = []
valLosses = []
valAccuracies = []

bestAcc = 0

for epoch in range(1, epochs+1):
    # Training loop
    model.train()
    totalTrain = 0
    correctTrain = 0

    with tqdm(total=len(train_loader), desc=F'Epoch {epoch}/{epochs+1} | Training') as pbar:
        for i, data in enumerate(train_loader):
            images, labels = data

            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            lTrain = criterion(outputs, labels)

            # Accuracy
            _, predicted = torch.max(outputs.data, 1)
            totalTrain += labels.size(0)
            correctTrain += (predicted == labels).sum().item()

            # Backward and optimize
            optimizer.zero_grad()
            lTrain.backward()
            optimizer.step()
            pbar.update()

    # Evaluation
    model.eval()
    with torch.no_grad():
        correctVal = 0
        totalVal = 0
        with tqdm(total=len(test_loader), desc=F'Epoch {epoch}/{epochs+1} | Test') as pbar:
            for i, data in enumerate(test_loader):
                images, labels = data

                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                lVal = criterion(outputs, labels)

                # Accuracy
                _, predicted = torch.max(outputs.data, 1)
                totalVal += labels.size(0)
                correctVal += (predicted == labels).sum().item()
                pbar.update()
            
    # Losses computation
    train_loss_epoch = lTrain.item()
    val_loss_epoch = lVal.item()
    trainLosses.append(train_loss_epoch)
    valLosses.append(val_loss_epoch)
    
    # Accuracies computation
    val_acc_epoch = correctVal / totalVal
    train_acc_epoch = correctTrain / totalTrain
    trainAccuracies.append(train_acc_epoch)
    valAccuracies.append(val_acc_epoch)

    if(val_acc_epoch > bestAcc):
        previousBestAcc = bestAcc
        bestAcc = val_acc_epoch
        print("\nVal Accuracy increased at epoch {}: {:.5f} --> {:.5f} | Saving model..".format(epoch, previousBestAcc, bestAcc))
        torch.save(model, F"resnet50_epoch-{epoch}_accT-{train_acc_epoch:.5f}_accV-{bestAcc:.5f}.pt")

    print('\nEpoch {}/{}:\n\tTrain Acc: {:.3f} (avg. {:.3f}) | Train Loss: {:.3f} (avg. {:.3f}) | \
        \n\tVal Acc  : {:.3f} (avg. {:.3f}) | Val Loss: {:.3f} (avg. {:.3f}) |'.format(epoch, epochs, 
                                                                                    train_acc_epoch*100,
                                                                                    np.average(trainAccuracies)*100,  
                                                                                    train_loss_epoch,
                                                                                    np.average(trainLosses),
                                                                                    val_acc_epoch*100,
                                                                                    np.average(valAccuracies)*100,
                                                                                    val_loss_epoch,
                                                                                    np.average(valLosses)))
    print("=========================================")